In [2]:
import generate_data as vaina
import models
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from evaluate import *


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

AttributeError: module 'numpy.core' has no attribute 'numerictypes'

Sacamos el conjunto de datos de entramiento y lo dividimos en 5% del subset para entrenamiento

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") #Define el dispositivo que PyTorch usará para ejecutar los cálculos.

data = vaina.Data(layer_size=6)
train, val, test = data.get_splits(['random_subsample'], [[0.25, 0.75]]) #internamente onehot
train_unshuffled = train.copy()
np.random.shuffle(train)

Number of samples: 132 train | 79 val | 318 test


c:\Users\juand\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but OneHotEncoder was fitted without feature names
  warnings.warn(


In [ ]:
data.samples

[[',..._.', 'w._,._', 'cr__._', 'V'],
 ['.,,_,.', 'w,_,__', '.cr._.', 'V'],
 [',__,._', 'w,._,.', '_,cr__', 'V'],
 [',___._', 'w.,_.,', ',..cr,', 'V'],
 ['__.,.,', 'w._,_.', '_..,cr', 'V'],
 [',,,__,', ',w._._', 'cr.,,_', 'V'],
 [',_..,_', '_w....', '.cr.__', 'V'],
 ['___.__', '_w_,_,', '..cr__', 'V'],
 ['_.,_._', '_w._.,', '...cr_', 'V'],
 ['_.,,.,', '.w...,', '__._cr', 'V'],
 [',,.,.,', '._w__,', 'cr_,_,', 'V'],
 ['_..__.', '__w_.,', ',cr,.,', 'V'],
 [',.._..', '_,w_..', '.,cr.,', 'V'],
 ['_.__,,', '_,w,_,', '__.cr.', 'V'],
 [',..,__', '.,w.,_', '.___cr', 'V'],
 [',,.._.', '...w.,', 'cr,,,,', 'V'],
 ['__,.._', '.,.w..', ',cr_.,', 'V'],
 ['_...__', '..,w__', ',_cr..', 'V'],
 ['__....', ',,,w._', '..,cr,', 'V'],
 ['..._.,', '.,_w_.', ',.__cr', 'V'],
 ['.._..,', '_.,_w,', 'cr__.,', 'V'],
 [',,,__.', '.__.w_', '_cr__,', 'V'],
 ['.___,_', '._,,w.', '__cr,,', 'V'],
 ['....__', ',..,w,', '__,cr.', 'V'],
 ['..,._,', '.__.w,', '__,,cr', 'V'],
 ['_,,_,.', '.__,_w', 'cr,_,.', 'V'],
 ['._,._,', 

Separamos todos los datos en x (variables), y (clasificacion) 

In [ ]:
x_train, y_train = data.get_x_y(train) #devuelve  self.np_data = self.one_hot_encode(self.samples_char_sep)
x_val, y_val = data.get_x_y(val)
x_test, y_test = data.get_x_y(test)

In [ ]:
x_train[0]
#print(x_train[:10])

array([1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0,
       0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,
       0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0,
       0.0], dtype=object)

In [ ]:
print(y_train[:10])


['V' 'C' 'A' 'V' 'V' 'V' 'V' 'V' 'V' 'V']


In [ ]:
train_original = data.reverse_encoding(data.get_x_y(train_unshuffled)[0])
val_original = data.reverse_encoding(x_val)
test_original = data.reverse_encoding(x_test)

print("--------train_original--------", '\n')
pos_train_ratio = get_stats_and_ratio(train_original)
print("--------test_original--------", '\n')
pos_test_ratio = get_stats_and_ratio(test_original)
print("--------val_original--------", '\n')
pos_val_ratio = get_stats_and_ratio(val_original)

--------train_original-------- 

Number of samples by case:
F3 dr:    F2-wu 4 | F2-uw 7 | F2-w 8 | F2-noop 12 | 
F3 cr:    F2-wu 6 | F2-uw 6 | F2-w 9 | F2-noop 6 | 
F3 r:    F2-wu 4 | F2-uw 4 | F2-w 11 | F2-noop 10 | 
F3 noop:    F2-wu 10 | F2-uw 9 | F2-w 12 | F2-noop 14 | 

Positive samples count: 56
Total samples count: 132
Positive class ratio: 0.42424242424242425
--------test_original-------- 

Number of samples by case:
F3 dr:    F2-wu 19 | F2-uw 15 | F2-w 19 | F2-noop 20 | 
F3 cr:    F2-wu 16 | F2-uw 14 | F2-w 17 | F2-noop 25 | 
F3 r:    F2-wu 21 | F2-uw 22 | F2-w 20 | F2-noop 25 | 
F3 noop:    F2-wu 20 | F2-uw 20 | F2-w 22 | F2-noop 23 | 

Positive samples count: 142
Total samples count: 318
Positive class ratio: 0.44654088050314467
--------val_original-------- 

Number of samples by case:
F3 dr:    F2-wu 2 | F2-uw 3 | F2-w 3 | F2-noop 3 | 
F3 cr:    F2-wu 3 | F2-uw 5 | F2-w 4 | F2-noop 4 | 
F3 r:    F2-wu 5 | F2-uw 4 | F2-w 5 | F2-noop 7 | 
F3 noop:    F2-wu 5 | F2-uw 6 | F2-w 

In [ ]:
print(test_original)

[['___._.', '__..,_', ',__.cr', 'V'], [',,__,,', '_._.,w', ',,,r_.', 'C'], [',,,._.', '__.,uw', '.cr_._', 'V'], ['_.._,,', '__,,uw', ',,.._r', 'C'], ['_,._..', '.._w.,', ',_,_dr', 'D'], ['__._.,', '_.,_uw', '_,.,_.', 'V'], ['.__,..', ',_wu._', 'r___..', 'C'], [',_..,,', ',_w._.', '.,_dr.', 'D'], ['_.,_,_', '__,,,,', '_cr._.', 'V'], ['.__.__', '.,w.__', '_._.r.', 'C'], [',._,__', ',,wu__', ',.,_cr', 'V'], ['.,,.,,', '_.,uw.', '.___._', 'V'], ['.,._,_', '_,.wu.', ',_,.dr', 'V'], [',_.,._', '.,,uw_', '__._cr', 'V'], [',_,,__', '.__uw.', ',r,,._', 'C'], [',.,,_,', '_,.w,.', ',_.,,.', 'V'], ['.___,_', ',,.,wu', ',r..__', 'C'], ['_,,_,,', '_,_.,,', '._,___', 'V'], ['_.,_,_', ',_w._,', ',dr,..', 'D'], ['._.,,.', '._,w,_', '_,.._,', 'V'], [',..,__', '.,w.,_', '.___cr', 'V'], [',._,._', ',.....', '_.r,_,', 'C'], [',,.._.', '...w.,', 'cr,,,,', 'V'], [',_,,,,', '..uw__', '_,__cr', 'V'], ['.__.._', '..,__.', ',_.,,.', 'V'], [',_._.,', ',__uw.', ',.__,,', 'V'], [',___._', 'w.,_.,', ',..cr,', 'V'], 

Mandamos las etiquetas codificadas al dispositivo del tensor

In [ ]:
# Crear un codificador de etiquetas
label_encoder = LabelEncoder()

#input_tensor = input_tensor.requires_grad_()  # Esto habilita el cálculo de gradientes

#cambio de torch.float a torch.long

# Ajustar el codificador a las etiquetas y transformarlas a num enteros
y_train = label_encoder.fit_transform(y_train)
y_val = label_encoder.transform(y_val)
y_test = label_encoder.transform(y_test)

y_trainP = y_train

#Un tensor es una estructura de datos similar a matrices o arrays, pero con soporte para operaciones avanzadas en GPU.
# Convertir las etiquetas a tensores de PyTorch
y_train = torch.tensor(y_train, dtype=torch.long).to(device)
y_val = torch.tensor(y_val, dtype=torch.long).to(device)
y_test = torch.tensor(y_test, dtype=torch.long).to(device)

#se mueven los datos a la CPU o GPU para que puedan ser procesados por PyTorch durante el entrenamiento o la inferencia

In [ ]:

print(set(y_train))



{tensor(3), tensor(1), tensor(1), tensor(1), tensor(1), tensor(2), tensor(3), tensor(3), tensor(1), tensor(3), tensor(1), tensor(2), tensor(3), tensor(3), tensor(2), tensor(3), tensor(2), tensor(1), tensor(3), tensor(3), tensor(3), tensor(3), tensor(3), tensor(1), tensor(0), tensor(3), tensor(1), tensor(3), tensor(2), tensor(1), tensor(3), tensor(3), tensor(3), tensor(3), tensor(0), tensor(3), tensor(3), tensor(2), tensor(3), tensor(3), tensor(0), tensor(3), tensor(3), tensor(1), tensor(3), tensor(1), tensor(0), tensor(3), tensor(3), tensor(3), tensor(3), tensor(1), tensor(3), tensor(3), tensor(3), tensor(3), tensor(3), tensor(1), tensor(3), tensor(3), tensor(3), tensor(3), tensor(1), tensor(1), tensor(3), tensor(3), tensor(3), tensor(3), tensor(1), tensor(3), tensor(3), tensor(1), tensor(2), tensor(3), tensor(1), tensor(1), tensor(3), tensor(3), tensor(2), tensor(3), tensor(2), tensor(1), tensor(3), tensor(1), tensor(3), tensor(3), tensor(3), tensor(2), tensor(3), tensor(2), tensor(0)

In [ ]:
x_train_cnn = data.to_conv_format(x_train)
x_val_cnn = data.to_conv_format(x_val)
x_test_cnn = data.to_conv_format(x_test)
for i in range(len(x_train_cnn)): #itereamos sobre cada tensor
    x_train_cnn[i] = x_train_cnn[i].to(device)
    x_val_cnn[i] = x_val_cnn[i].to(device)
    x_test_cnn[i] = x_test_cnn[i].to(device)

In [ ]:
x_train_cnn[0].shape #devuelve un par que indica (num de muestras, numero de funciones en total = 3*numMuestras)

torch.Size([132, 48])

In [ ]:
x_train_cnn[0][2] #[x][y] accedes fx de la muestra y / max [2][25] ya que hay 3 funciones y este el ejemplo tenemos 26 muestras de train

tensor([0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

x_train_cnn está representado como una matriz tridimensional en donde cada fila es la función y cada columna es la muestra

In [ ]:
num_experiments = 6 #cuantas veces ejecutaremos el entrenamiento, pero con dif pesos, entrena, parame, semillas
epochs = 200 #época es una iteración completa sobre todo el conjunto de entrenamiento
early_stopping_limit = 100 #si el cjto de val no ve una mejora durante 100 epocas consecutivas, el entrenam se para. Evitamos sobreajuste

experiment_name="25per"

#el número de épocas determina cuántas veces el modelo va a ajustarse a los datos y aprender de ellos. Sin embargo, no siempre se necesita un número tan alto; por eso también se utilizan estrategias como el early stopping para evitar el sobreajuste.

In [ ]:
def train_models(constructor, x_train, x_val, x_test, weight_decay, *argv):
    accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    wrong_preds = []
    best_accuracy = 0
    
    train_losses = []
    val_losses = []
    train_accs = []
    val_accs = []

    for i in range(num_experiments):
        model = constructor(*argv)
        model.to(device)

        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), weight_decay=weight_decay)

        train_losses.append([])
        val_losses.append([])
        train_accs.append([])
        val_accs.append([])
        
        best_acc = 0

        early_stopping_cnt = 0

        for epoch in range(1, epochs + 1):
            train_loss, train_acc = models.train_epoch(model, x_train, y_train, criterion, optimizer, epoch, 10, verbose=False)
            val_loss, val_acc =models.eval_epoch(model, x_val, y_val, criterion, 'Validation', verbose=False)

            
            train_losses[-1].append(train_loss)
            val_losses[-1].append(val_loss)
            train_accs[-1].append(train_acc)
            val_accs[-1].append(val_acc)
            
            model_name = constructor.__name__[:constructor.__name__.find('_')]
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), f'./{model_name}_model_TEMP_' + experiment_name)    
                early_stopping_cnt = 0
            else:
                early_stopping_cnt += 1

            if early_stopping_cnt >= early_stopping_limit:
                break


        model.load_state_dict(torch.load(f'./{model_name}_model_TEMP_' + experiment_name))
    
        accuracies.append(get_accuracy_by_cases(model, x_test, y_test, test_original))
        precisions.append(get_precision_by_cases(model, x_test, y_test, test_original))
        recalls.append(get_recall_by_cases(model, x_test, y_test, test_original))
        f1_scores.append(get_f1_by_cases(precisions[-1], recalls[-1]))
        
        wrong_preds.append((f"Experimento {i}", get_wrong_predictions(model, x_test, y_test, test_original)))

        if accuracies[-1]['Overall'] > best_accuracy:
            torch.save(model.state_dict(), f'./best_{model_name}_model_' + experiment_name)    
            best_accuracy = accuracies[-1]['Overall']

        print(i + 1, "/", num_experiments, "models trained | Current model test accuracy:", accuracies[-1]['Overall'])
        print(i + 1, "/", num_experiments, "models trained | Current model test precision:", precisions[-1]['Overall'])
        print(i + 1, "/", num_experiments, "models trained | Current model test recall:", recalls[-1]['Overall'])
        print(i + 1, "/", num_experiments, "models trained | Current model test f1:", f1_scores[-1]['Overall'])


    
    return accuracies, precisions, recalls, f1_scores, wrong_preds, [train_losses, val_losses, train_accs, val_accs], model
    #return accuracies, precisions, wrong_preds, [train_losses, val_losses, train_accs, val_accs]

In [ ]:
cnn_accuracies, cnn_precisions, cnn_recall, cnn_f1, cnn_wrong_preds, cnn_epoch_stats, model = train_models(
    models.CNN_Model, x_train_cnn, x_val_cnn, x_test_cnn, 0.0001, data, 64, 128, 4, -1, 'keras')

C:\Users\juand\AppData\Local\Temp\ipykernel_20844\2901190800.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}_model_TEM

1 / 6 models trained | Current model test accuracy: 0.8962264150943396
1 / 6 models trained | Current model test precision: 0.7417993450767841
1 / 6 models trained | Current model test recall: 0.7963650932400932
1 / 6 models trained | Current model test f1: 0.7681143704686458
2 / 6 models trained | Current model test accuracy: 0.9308176100628931
2 / 6 models trained | Current model test precision: 0.8531725686942299
2 / 6 models trained | Current model test recall: 0.943939393939394
2 / 6 models trained | Current model test f1: 0.8962637989886133
3 / 6 models trained | Current model test accuracy: 0.8962264150943396
3 / 6 models trained | Current model test precision: 0.7756049682457311
3 / 6 models trained | Current model test recall: 0.8463650932400932
3 / 6 models trained | Current model test f1: 0.8094415388474357
4 / 6 models trained | Current model test accuracy: 0.9528301886792453
4 / 6 models trained | Current model test precision: 0.8722531680824397
4 / 6 models trained | Curr

In [ ]:
cnn_wrong_preds

[('Experimento 0',
  ([(['_,_,._', '..,,uw', 'dr__,.', 'A'], 'V'),
    (['_.,.__', '_,_,uw', '_,_dr,', 'A'], 'D'),
    (['.,._,.', '.__,uw', ',_dr_,', 'A'], 'V'),
    ([',.,_._', '.__,uw', ',,._dr', 'A'], 'V')],
   [],
   [],
   [(['.,._,_', '_,.wu.', ',_,.dr', 'V'], 'D'),
    (['__.._.', '__wu..', ',_dr..', 'V'], 'D'),
    (['___._.', 'wu__,,', 'dr,_,_', 'V'], 'A'),
    ([',.,,,,', '_,,,wu', 'dr_...', 'V'], 'A'),
    (['__._,_', '_.wu,.', ',,_dr.', 'V'], 'A'),
    (['__,,,_', '_wu_,,', '..dr._', 'V'], 'D'),
    (['_,,,_.', ',,.wu_', '_,_dr_', 'V'], 'D'),
    (['._,..,', ',_,wu_', '.,dr_,', 'V'], 'D'),
    (['__,.._', '._wu.,', 'dr,,__', 'V'], 'D'),
    ([',.,._.', '.wu.__', '..,dr,', 'V'], 'D'),
    ([',,_.._', ',,..wu', ',__dr.', 'V'], 'D'),
    ([',.,,.,', ',,,_wu', '._,_dr', 'V'], 'D'),
    (['._._,.', '.,,.wu', '_dr__.', 'V'], 'D'),
    (['..,,_.', 'wu.,_,', '_dr,,,', 'V'], 'D'),
    (['_..__,', '._wu,.', '.dr,,.', 'V'], 'D'),
    ([',_,,,,', 'wu.,,,', '.,_dr_', 'V'], 'A'),
    ([

In [ ]:
model.eval()

output = model(x_test_cnn[0], x_test_cnn[1], x_test_cnn[2])
y_pred = torch.argmax(torch.softmax(output, dim=1), dim=1)
print(y_pred)

tensor([3, 1, 3, 1, 2, 3, 1, 2, 3, 1, 3, 3, 1, 3, 1, 3, 1, 3, 2, 3, 3, 1, 3, 3,
        3, 3, 3, 3, 3, 1, 3, 1, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 1,
        1, 0, 0, 1, 3, 3, 3, 0, 3, 2, 3, 3, 1, 1, 3, 0, 0, 3, 3, 3, 3, 3, 3, 1,
        1, 3, 1, 3, 2, 3, 2, 0, 2, 1, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 1, 2, 3, 1,
        1, 1, 1, 3, 1, 0, 3, 3, 2, 3, 2, 1, 0, 1, 0, 3, 3, 1, 3, 0, 1, 2, 1, 1,
        3, 3, 3, 3, 2, 3, 2, 1, 3, 3, 3, 1, 2, 3, 1, 1, 1, 1, 0, 3, 3, 3, 1, 1,
        2, 3, 2, 3, 3, 3, 3, 0, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 2, 3, 3,
        0, 1, 3, 2, 3, 3, 3, 1, 2, 1, 3, 1, 1, 0, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3,
        1, 3, 3, 1, 1, 2, 3, 2, 2, 2, 3, 1, 3, 1, 2, 3, 2, 3, 2, 0, 3, 3, 3, 3,
        3, 3, 3, 3, 1, 3, 1, 3, 3, 3, 1, 3, 0, 1, 2, 1, 3, 3, 3, 0, 3, 1, 3, 3,
        2, 2, 1, 1, 3, 3, 1, 2, 1, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 3, 2, 2, 3,
        3, 1, 2, 3, 0, 3, 1, 1, 3, 2, 1, 1, 0, 3, 3, 3, 3, 3, 3, 3, 1, 1, 3, 1,
        2, 3, 3, 2, 3, 3, 0, 3, 1, 3, 3,

In [ ]:
x_train_lstm = data.to_lstm_format(x_train)
x_val_lstm = data.to_lstm_format(x_val)
x_test_lstm = data.to_lstm_format(x_test)
for i in range(len(x_train_lstm)):
    x_train_lstm[i] = x_train_lstm[i].to(device)
    x_val_lstm[i] = x_val_lstm[i].to(device)
    x_test_lstm[i] = x_test_lstm[i].to(device)

In [ ]:
lstm_accuracies, lstm_precision, lstm_recall, lstm_f1, lstm_wrong_preds, lstm_epoch_stats, model = train_models(models.LSTM_Model, x_train_lstm, x_val_lstm, x_test_lstm, 0.0001, data, 16, 32, 8, 'keras')

Número de características de entrada por paso de tiempo 8


C:\Users\juand\AppData\Local\Temp\ipykernel_26492\2901190800.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}_model_TEM

1 / 6 models trained | Current model test accuracy: 0.9213836477987422
1 / 6 models trained | Current model test precision: 0.8064600840336134
1 / 6 models trained | Current model test recall: 0.8730113636363637
1 / 6 models trained | Current model test f1: 0.8384171325535148
Número de características de entrada por paso de tiempo 8
2 / 6 models trained | Current model test accuracy: 0.9056603773584906
2 / 6 models trained | Current model test precision: 0.7633612240019125
2 / 6 models trained | Current model test recall: 0.8307036713286713
2 / 6 models trained | Current model test f1: 0.7956099819848665
Número de características de entrada por paso de tiempo 8
3 / 6 models trained | Current model test accuracy: 0.89937106918239
3 / 6 models trained | Current model test precision: 0.7166990416990418
3 / 6 models trained | Current model test recall: 0.7566215034965035
3 / 6 models trained | Current model test f1: 0.7361193845469112
Número de características de entrada por paso de tiempo

In [ ]:
model.eval()

output = model(x_train_lstm[0], x_train_lstm[1], x_train_lstm[2])
y_pred = torch.argmax(torch.softmax(output, dim=1), dim=1)
print(y_pred)

tensor([3, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 0, 3, 2, 1, 1, 3, 3, 2, 3, 3, 3,
        2, 3, 3, 3, 3, 2, 3, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 0, 2, 1, 3, 2,
        3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 2, 1, 0, 2, 1, 3, 3, 1, 1, 3, 2, 3, 3,
        3, 1, 1, 1, 2, 0, 3, 3, 3, 1, 1, 3, 1, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3,
        3, 2, 2, 3, 3, 3, 3, 3, 3, 2, 1, 3, 3, 1, 3, 1, 1, 3, 3, 3, 2, 3, 1, 1,
        1, 2, 1, 3, 0, 1, 3, 2, 1, 3, 3, 1])


In [ ]:
x_train_normal = data.to_conv_format(x_train)
x_val_normal = data.to_conv_format(x_val)
x_test_normal = data.to_conv_format(x_test)
for i in range(len(x_train_normal)):
    x_train_normal[i] = x_train_normal[i].to(device)
    x_val_normal[i] = x_val_normal[i].to(device)
    x_test_normal[i] = x_test_normal[i].to(device)

In [ ]:
deepset_accuracies, deepset_procision, deepset_recall, deepset_f1, deepset_wrong_preds, deepset_epoch_stats, model = train_models(models.DEEPSET_Model, x_train_normal, x_val_normal, x_test_normal, 0.005, data, 128, 32, 8, 'keras')

C:\Users\juand\AppData\Local\Temp\ipykernel_26492\2901190800.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}_model_TEM

1 / 6 models trained | Current model test accuracy: 0.8930817610062893
1 / 6 models trained | Current model test precision: 0.7528981677237492
1 / 6 models trained | Current model test recall: 0.788304244774833
1 / 6 models trained | Current model test f1: 0.7701945139543722
2 / 6 models trained | Current model test accuracy: 0.8364779874213837
2 / 6 models trained | Current model test precision: 0.6938981734299972
2 / 6 models trained | Current model test recall: 0.792436974789916
2 / 6 models trained | Current model test f1: 0.7399011858444707
3 / 6 models trained | Current model test accuracy: 0.8616352201257862
3 / 6 models trained | Current model test precision: 0.7207528436475805
3 / 6 models trained | Current model test recall: 0.7739873780837636
3 / 6 models trained | Current model test f1: 0.7464221482647343
4 / 6 models trained | Current model test accuracy: 0.8647798742138365
4 / 6 models trained | Current model test precision: 0.7266423939121307
4 / 6 models trained | Curre

In [ ]:
deepsetv2_accuracies, deepsetv2_procision, deepsetv2_recall, deepsetv2_f1, deepsetv2_wrong_preds, deepsetv2_epoch_stats, model = train_models(models.DEEPSETV2_Model, x_train_normal, x_val_normal, x_test_normal, 0.0001, data, 32, 8, 'keras')

C:\Users\juand\AppData\Local\Temp\ipykernel_26492\2901190800.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}_model_TEM

1 / 6 models trained | Current model test accuracy: 0.6352201257861635
1 / 6 models trained | Current model test precision: 0.3571575435009928
1 / 6 models trained | Current model test recall: 0.5680215617715618
1 / 6 models trained | Current model test f1: 0.4385598085857391
2 / 6 models trained | Current model test accuracy: 0.5849056603773585
2 / 6 models trained | Current model test precision: 0.28605200945626474
2 / 6 models trained | Current model test recall: 0.3640005827505828
2 / 6 models trained | Current model test f1: 0.3203528434078561
3 / 6 models trained | Current model test accuracy: 0.6823899371069182
3 / 6 models trained | Current model test precision: 0.32312911184210524
3 / 6 models trained | Current model test recall: 0.3793859649122807
3 / 6 models trained | Current model test f1: 0.34900503617326817
4 / 6 models trained | Current model test accuracy: 0.7264150943396226
4 / 6 models trained | Current model test precision: 0.5861132995485905
4 / 6 models trained | 

In [ ]:
model.eval()

output = model(x_train_normal[0], x_train_normal[1], x_train_normal[2])
y_pred = torch.argmax(torch.softmax(output, dim=1), dim=1)
print(y_pred)

tensor([3, 1, 2, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 0, 3, 2, 1, 1, 3, 3, 2, 3, 3, 3,
        2, 3, 3, 3, 3, 2, 3, 2, 3, 3, 2, 3, 3, 2, 3, 2, 3, 3, 0, 0, 2, 1, 3, 2,
        3, 3, 3, 3, 3, 3, 0, 2, 3, 3, 0, 2, 1, 0, 2, 1, 3, 3, 0, 1, 3, 2, 3, 3,
        3, 1, 0, 1, 2, 0, 3, 3, 3, 1, 1, 3, 1, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3,
        3, 2, 2, 3, 3, 3, 3, 3, 3, 2, 0, 3, 3, 1, 3, 1, 0, 3, 3, 3, 2, 3, 1, 1,
        1, 2, 1, 3, 1, 1, 3, 2, 1, 3, 3, 1])


In [ ]:
feedforward_accuracies, feedforward_precision, feedforward_recall, feedforward_f1, feedforward_wrong_preds, feedforward_epoch_stats, model = train_models(models.FEEDFORWARD_Model, x_train_normal, x_val_normal, x_test_normal, 0.0001, data, 128, 32, 8, 'keras')

C:\Users\juand\AppData\Local\Temp\ipykernel_26492\2901190800.py:52: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f'./{model_name}_model_TEM

1 / 6 models trained | Current model test accuracy: 0.7358490566037735
1 / 6 models trained | Current model test precision: 0.6270269714370642
1 / 6 models trained | Current model test recall: 0.7583333333333333
1 / 6 models trained | Current model test f1: 0.6864574532739768
2 / 6 models trained | Current model test accuracy: 0.7452830188679245
2 / 6 models trained | Current model test precision: 0.5891509685196087
2 / 6 models trained | Current model test recall: 0.6632717709640786
2 / 6 models trained | Current model test f1: 0.6240180634476459
3 / 6 models trained | Current model test accuracy: 0.7515723270440252
3 / 6 models trained | Current model test precision: 0.5867193108399138
3 / 6 models trained | Current model test recall: 0.6437275295326144
3 / 6 models trained | Current model test f1: 0.6139027873511508
4 / 6 models trained | Current model test accuracy: 0.7327044025157232
4 / 6 models trained | Current model test precision: 0.49405298263773045
4 / 6 models trained | Cu

In [ ]:
model.eval()

output = model(x_train_normal[0], x_train_normal[1], x_train_normal[2])
y_pred = torch.argmax(torch.softmax(output, dim=1), dim=1)
print(y_pred)

tensor([3, 1, 0, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 0, 3, 2, 1, 1, 3, 3, 2, 3, 3, 3,
        2, 3, 3, 3, 3, 2, 3, 2, 3, 3, 2, 3, 3, 2, 3, 3, 3, 3, 3, 0, 2, 1, 3, 2,
        3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 1, 2, 1, 0, 2, 1, 3, 3, 1, 1, 3, 2, 3, 3,
        3, 1, 1, 1, 2, 0, 3, 3, 3, 1, 1, 3, 1, 3, 1, 1, 3, 3, 1, 3, 3, 3, 3, 3,
        3, 2, 2, 3, 3, 3, 3, 3, 3, 2, 1, 3, 3, 1, 3, 1, 1, 3, 3, 3, 2, 3, 1, 1,
        1, 2, 1, 3, 0, 1, 3, 2, 1, 3, 3, 1])


## Accuracy breakdown by cases for all the models

In [ ]:
top_half = int(num_experiments / 2)
best_cnn_accs = filter_top_k_accuracies(cnn_accuracies, top_half)
best_lstm_accs = filter_top_k_accuracies(lstm_accuracies, top_half)
best_deepset_accs = filter_top_k_accuracies(deepset_accuracies, top_half)
best_deepsetv2_accs = filter_top_k_accuracies(deepsetv2_accuracies, top_half)
best_feedforward_accs = filter_top_k_accuracies(feedforward_accuracies, top_half)

In [ ]:
all_accuracies = [cnn_accuracies, lstm_accuracies, deepset_accuracies, deepsetv2_accuracies, feedforward_accuracies]
model_names = ['CNN', 'LSTM', 'DeepSet(like in paper)', 'DeepSet(sum at start)', 'Feedforward']
get_stats_df(all_accuracies, model_names, test_original)

,Overall,cr_noop,cr_uw,cr_w,cr_wu,dr_noop,dr_uw,dr_w,dr_wu,noop_noop,noop_uw,noop_w,noop_wu,r_noop,r_uw,r_w,r_wu
CNN,93.0%,100.0%,100.0%,100.0%,100.0%,100.0%,71.1%,98.2%,25.4%,100.0%,100.0%,100.0%,100.0%,99.3%,90.9%,100.0%,94.4%
LSTM,90.6%,100.0%,100.0%,100.0%,100.0%,99.2%,44.4%,84.2%,7.0%,100.0%,100.0%,100.0%,100.0%,99.3%,99.2%,100.0%,98.4%
DeepSet(like in paper),85.7%,100.0%,100.0%,100.0%,100.0%,83.3%,31.1%,82.5%,8.8%,100.0%,100.0%,100.0%,100.0%,88.0%,78.8%,90.8%,92.1%
DeepSet(sum at start),65.5%,98.7%,100.0%,99.0%,100.0%,55.0%,13.3%,52.6%,21.1%,98.6%,99.2%,100.0%,99.2%,34.0%,24.2%,29.2%,27.8%
Feedforward,72.3%,98.0%,97.6%,100.0%,99.0%,56.7%,15.6%,65.8%,17.5%,98.6%,94.2%,99.2%,96.7%,61.3%,47.7%,58.3%,43.7%
Sample count,318,25,14,17,16,20,15,19,19,23,20,22,20,25,22,20,21


## Accuracy breakdown by cases for top 50% of models

In [ ]:
best_accuracies = [best_cnn_accs, best_lstm_accs, best_deepset_accs, best_deepsetv2_accs, best_feedforward_accs]
model_names = ['CNN', 'LSTM', 'DeepSet(like in paper)', 'DeepSet(sum at start)', 'Feedforward']
collapsed_cases = ['dr', 'r', 'cr', 'noop']
get_stats_df(best_accuracies, model_names, test_original, collapsed_cases)

,Overall,f3_cr,f3_dr,f3_noop,f3_r
CNN,94.2%,100.0%,76.3%,100.0%,98.9%
LSTM,91.6%,100.0%,65.3%,100.0%,98.5%
DeepSet(like in paper),87.8%,100.0%,57.5%,100.0%,91.3%
DeepSet(sum at start),71.1%,98.6%,27.9%,98.4%,58.0%
Feedforward,74.4%,99.1%,43.4%,96.5%,58.7%
Sample count,318,72,73,85,88


In [ ]:
best_accuracies = [best_cnn_accs, best_lstm_accs, best_deepset_accs, best_deepsetv2_accs, best_feedforward_accs]
model_names = ['CNN', 'LSTM', 'DeepSet(like in paper)', 'DeepSet(sum at start)', 'Feedforward']
collapsed_cases = ['r', 'cr', 'noop']
get_stats_df(best_accuracies, model_names, test_original, collapsed_cases)

,Overall,dr_noop,dr_uw,dr_w,dr_wu,f3_cr,f3_noop,f3_r
CNN,94.2%,100.0%,75.6%,100.0%,28.1%,100.0%,100.0%,98.9%
LSTM,91.6%,100.0%,60.0%,87.7%,10.5%,100.0%,100.0%,98.5%
DeepSet(like in paper),87.8%,90.0%,33.3%,89.5%,10.5%,100.0%,100.0%,91.3%
DeepSet(sum at start),71.1%,45.0%,13.3%,40.4%,8.8%,98.6%,98.4%,58.0%
Feedforward,74.4%,63.3%,22.2%,70.2%,12.3%,99.1%,96.5%,58.7%
Sample count,318,20,15,19,19,72,85,88


In [ ]:
get_stats_per_model(cnn_accuracies, ['CNN #' + str(i) for i in range(len(cnn_accuracies))], test_original, ['cr', 'dr', 'noop', 'r'])

,Overall,f3_cr,f3_dr,f3_noop,f3_r
CNN #0,93.1%,100.0%,69.9%,100.0%,100.0%
CNN #1,92.8%,100.0%,68.5%,100.0%,100.0%
CNN #2,93.4%,100.0%,71.2%,100.0%,100.0%
CNN #3,94.0%,100.0%,74.0%,100.0%,100.0%
CNN #4,95.3%,100.0%,83.6%,100.0%,96.6%
CNN #5,89.6%,100.0%,78.1%,100.0%,80.7%
Sample count,318,72,73,85,88


In [ ]:
get_stats_per_model(lstm_accuracies, ['LSTM #' + str(i) for i in range(len(lstm_accuracies))], test_original, ['cr', 'dr', 'noop', 'r'])

,Overall,f3_cr,f3_dr,f3_noop,f3_r
LSTM #0,92.1%,100.0%,65.8%,100.0%,100.0%
LSTM #1,90.6%,100.0%,58.9%,100.0%,100.0%
LSTM #2,89.9%,100.0%,56.2%,100.0%,100.0%
LSTM #3,90.9%,100.0%,65.8%,100.0%,95.5%
LSTM #4,88.4%,100.0%,49.3%,100.0%,100.0%
LSTM #5,91.8%,100.0%,64.4%,100.0%,100.0%
Sample count,318,72,73,85,88


In [ ]:
get_stats_per_model(deepset_accuracies, ['DeepSet(like in paper) #' + str(i) for i in range(len(deepset_accuracies))], test_original, ['cr', 'dr', 'noop', 'r'])

,Overall,f3_cr,f3_dr,f3_noop,f3_r
DeepSet(like in paper) #0,89.3%,100.0%,58.9%,100.0%,95.5%
DeepSet(like in paper) #1,83.6%,100.0%,54.8%,100.0%,78.4%
DeepSet(like in paper) #2,86.2%,100.0%,53.4%,100.0%,88.6%
DeepSet(like in paper) #3,86.5%,100.0%,56.2%,100.0%,87.5%
DeepSet(like in paper) #4,87.7%,100.0%,57.5%,100.0%,90.9%
DeepSet(like in paper) #5,80.8%,100.0%,37.0%,100.0%,83.0%
Sample count,318,72,73,85,88


In [ ]:
get_stats_per_model(deepsetv2_accuracies, ['DeepSet(sum at start) #' + str(i) for i in range(len(deepsetv2_accuracies))], test_original, ['cr', 'dr', 'noop', 'r'])

,Overall,f3_cr,f3_dr,f3_noop,f3_r
DeepSet(sum at start) #0,63.5%,100.0%,61.6%,100.0%,0.0%
DeepSet(sum at start) #1,58.5%,100.0%,39.7%,100.0%,0.0%
DeepSet(sum at start) #2,68.2%,100.0%,1.4%,100.0%,67.0%
DeepSet(sum at start) #3,72.6%,100.0%,38.4%,96.5%,55.7%
DeepSet(sum at start) #4,57.9%,100.0%,37.0%,100.0%,0.0%
DeepSet(sum at start) #5,72.3%,95.8%,43.8%,98.8%,51.1%
Sample count,318,72,73,85,88


In [ ]:
get_stats_per_model(feedforward_accuracies, ['Feedforward #' + str(i) for i in range(len(feedforward_accuracies))], test_original, ['cr', 'dr', 'noop', 'r'])

,Overall,f3_cr,f3_dr,f3_noop,f3_r
Feedforward #0,73.6%,98.6%,50.7%,97.6%,48.9%
Feedforward #1,74.5%,98.6%,43.8%,91.8%,63.6%
Feedforward #2,75.2%,100.0%,35.6%,100.0%,63.6%
Feedforward #3,73.3%,95.8%,31.5%,97.6%,65.9%
Feedforward #4,70.1%,98.6%,46.6%,98.8%,38.6%
Feedforward #5,67.0%,100.0%,34.2%,97.6%,37.5%
Sample count,318,72,73,85,88


# Best performing models

In [ ]:
best_lstm_model = models.LSTM_Model(data, 16, 32, 8).to(device)
best_lstm_model.load_state_dict(torch.load('best_LSTM_model_' + experiment_name))
best_cnn_model = models.CNN_Model(data, 64, 128, 4, -1,).to(device)
best_cnn_model.load_state_dict(torch.load('best_CNN_model_' + experiment_name))
best_deepset_model = models.DEEPSET_Model(data).to(device)
best_deepset_model.load_state_dict(torch.load('best_DEEPSET_model_' + experiment_name))
best_deepsetv2_model = models.DEEPSETV2_Model(data).to(device)
best_deepsetv2_model.load_state_dict(torch.load('best_DEEPSETV2_model_' + experiment_name))
best_feedforward_model = models.FEEDFORWARD_Model(data).to(device)
best_feedforward_model.load_state_dict(torch.load('best_FEEDFORWARD_model_' + experiment_name))

Número de características de entrada por paso de tiempo 8


C:\Users\juand\AppData\Local\Temp\ipykernel_26492\2902303673.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_lstm_model.load_state_dict(torch.load('best_LSTM_model_

<All keys matched successfully>

## Wrong predictions for best performing CNN model

In [ ]:
print_wrong_preds_bycases(get_wrong_predictions_bycases(best_cnn_model, x_test_cnn, y_test, test_original), k=10)

In [ ]:
print_wrong_preds(get_wrong_predictions(best_cnn_model, x_test_cnn, y_test, test_original), k=10)

Top 4 false positives: 

Muestra ['_,,_,_', '..,uw.', '.,dr,_', 'A'] | Prediccion V
Muestra ['__._,,', '.,uw,,', '_.dr_,', 'A'] | Prediccion D
Muestra [',_,__,', '.,uw._', '_.._dr', 'A'] | Prediccion V
Muestra [',.,_._', '.__,uw', ',,._dr', 'A'] | Prediccion V
Top 3 false positives: 

Muestra ['__.__,', ',_,wu,', '...,,r', 'C'] | Prediccion V
Muestra [',.,.__', ',wu_,,', '__.,,r', 'C'] | Prediccion V
Muestra [',_,_,.', 'wu___,', '_.,_r,', 'C'] | Prediccion V
Top 0 false positives: 

Top 8 false positives: 

Muestra ['.,._,_', '_,.wu.', ',_,.dr', 'V'] | Prediccion D
Muestra ['__.._.', '__wu..', ',_dr..', 'V'] | Prediccion D
Muestra ['_,__,_', '.wu_.,', '.dr_,,', 'V'] | Prediccion D
Muestra ['__._,_', '_.wu,.', ',,_dr.', 'V'] | Prediccion A
Muestra ['__,.._', '._wu.,', 'dr,,__', 'V'] | Prediccion D
Muestra [',.,._.', '.wu.__', '..,dr,', 'V'] | Prediccion A
Muestra [',,_.._', ',,..wu', ',__dr.', 'V'] | Prediccion D
Muestra ['._._,.', '.,,.wu', '_dr__.', 'V'] | Prediccion D




## Wrong predictions for best performing LSTM model

In [ ]:
print_wrong_preds_bycases(get_wrong_predictions_bycases(best_lstm_model, x_test_lstm, y_test, test_original), k=10)

In [ ]:
print_wrong_preds(get_wrong_predictions(best_lstm_model, x_test_lstm, y_test, test_original), k=10)

Top 6 false positives: 

Muestra ['_,_,._', '..,,uw', 'dr__,.', 'A'] | Prediccion D
Muestra [',_..,.', '.,_uw,', '_dr__,', 'A'] | Prediccion D
Muestra [',..,,_', ',_,_uw', '.dr..,', 'A'] | Prediccion D
Muestra ['_.,.__', '_,_,uw', '_,_dr,', 'A'] | Prediccion D
Muestra ['.,._,.', '.__,uw', ',_dr_,', 'A'] | Prediccion D
Muestra [',.,_._', '.__,uw', ',,._dr', 'A'] | Prediccion D
Top 0 false positives: 

Top 0 false positives: 

Top 10 false positives: 

Muestra ['.,._,_', '_,.wu.', ',_,.dr', 'V'] | Prediccion D
Muestra ['__.._.', '__wu..', ',_dr..', 'V'] | Prediccion A
Muestra ['_,__,_', '.wu_.,', '.dr_,,', 'V'] | Prediccion A
Muestra ['___._.', 'wu__,,', 'dr,_,_', 'V'] | Prediccion D
Muestra [',.,,,,', '_,,,wu', 'dr_...', 'V'] | Prediccion D
Muestra ['__._,_', '_.wu,.', ',,_dr.', 'V'] | Prediccion A
Muestra ['__,,,_', '_wu_,,', '..dr._', 'V'] | Prediccion D
Muestra ['_,,,_.', ',,.wu_', '_,_dr_', 'V'] | Prediccion D
Muestra ['._,..,', ',_,wu_', '.,dr_,', 'V'] | Prediccion D
Muestra ['__,.

## Wrong predictions for best performing DeepSets V1 model

In [ ]:
print_wrong_preds_bycases(get_wrong_predictions_bycases(best_deepset_model, x_test_normal, y_test, test_original), k=10)

In [ ]:
print_wrong_preds(get_wrong_predictions(best_deepset_model, x_test_normal, y_test, test_original), k=10)

Top 10 false positives: 

Muestra ['_,_,._', '..,,uw', 'dr__,.', 'A'] | Prediccion V
Muestra [',,__..', '.,uw._', '._.dr,', 'A'] | Prediccion D
Muestra [',_.,,_', 'uw,..,', ',dr_,.', 'A'] | Prediccion C
Muestra [',_..,.', '.,_uw,', '_dr__,', 'A'] | Prediccion D
Muestra ['__._,,', '.,uw,,', '_.dr_,', 'A'] | Prediccion D
Muestra [',_,__,', '.,uw._', '_.._dr', 'A'] | Prediccion D
Muestra [',..,,_', ',_,_uw', '.dr..,', 'A'] | Prediccion D
Muestra ['_.,.__', '_,_,uw', '_,_dr,', 'A'] | Prediccion D
Muestra ['.,._,.', '.__,uw', ',_dr_,', 'A'] | Prediccion D
Muestra [',.,_._', '.__,uw', ',,._dr', 'A'] | Prediccion D
Top 7 false positives: 

Muestra ['._____', ',._.w,', 'dr,,_.', 'D'] | Prediccion V
Muestra ['.__.__', ',.,,._', '...,r.', 'C'] | Prediccion V
Muestra ['__,..,', ',.._,.', '__r.__', 'C'] | Prediccion V
Muestra ['__.__,', '_uw.,,', '_.,,r,', 'C'] | Prediccion A
Muestra [',,.,_,', '___,,.', '___,r,', 'C'] | Prediccion V
Muestra ['.,__.,', ',.,,._', ',_dr,.', 'D'] | Prediccion V
Muest

## Wrong predictions for best performing DeepSets V2 model

In [ ]:
print_wrong_preds_bycases(get_wrong_predictions_bycases(best_deepsetv2_model, x_test_normal, y_test, test_original), k=10)

In [ ]:
print_wrong_preds(get_wrong_predictions(best_deepsetv2_model, x_test_normal, y_test, test_original), k=10)

Top 10 false positives: 

Muestra ['_,,_,_', '..,uw.', '.,dr,_', 'A'] | Prediccion D
Muestra [',_.,,_', 'uw,..,', ',dr_,.', 'A'] | Prediccion V
Muestra ['.,,,_.', '_uw._.', ',.,_dr', 'A'] | Prediccion D
Muestra [',_..,.', '.,_uw,', '_dr__,', 'A'] | Prediccion D
Muestra ['__._,,', '.,uw,,', '_.dr_,', 'A'] | Prediccion D
Muestra ['____,,', '_uw._.', ',_.dr,', 'A'] | Prediccion D
Muestra [',_,__,', '.,uw._', '_.._dr', 'A'] | Prediccion V
Muestra ['_.._,_', 'uw_,__', 'dr..,,', 'A'] | Prediccion D
Muestra [',..,,_', ',_,_uw', '.dr..,', 'A'] | Prediccion D
Muestra ['_.,.__', '_,_,uw', '_,_dr,', 'A'] | Prediccion D
Top 10 false positives: 

Muestra [',,__,,', '_._.,w', ',,,r_.', 'C'] | Prediccion V
Muestra ['_.._,,', '__,,uw', ',,.._r', 'C'] | Prediccion V
Muestra ['_,._..', '.._w.,', ',_,_dr', 'D'] | Prediccion A
Muestra [',_..,,', ',_w._.', '.,_dr.', 'D'] | Prediccion V
Muestra [',_,,__', '.__uw.', ',r,,._', 'C'] | Prediccion V
Muestra ['.___,_', ',,.,wu', ',r..__', 'C'] | Prediccion V
Mues

## Wrong predictions for best performing Feedforward model

In [ ]:
print_wrong_preds_bycases(get_wrong_predictions_bycases(best_feedforward_model, x_test_normal, y_test, test_original), k=10)

In [ ]:
print_wrong_preds(get_wrong_predictions(best_feedforward_model, x_test_normal, y_test, test_original), k=10)

Top 10 false positives: 

Muestra ['_,,_,_', '..,uw.', '.,dr,_', 'A'] | Prediccion D
Muestra ['_,_,._', '..,,uw', 'dr__,.', 'A'] | Prediccion V
Muestra [',,__..', '.,uw._', '._.dr,', 'A'] | Prediccion D
Muestra [',_.,,_', 'uw,..,', ',dr_,.', 'A'] | Prediccion D
Muestra ['.,,,_.', '_uw._.', ',.,_dr', 'A'] | Prediccion C
Muestra [',_..,.', '.,_uw,', '_dr__,', 'A'] | Prediccion D
Muestra ['__._,,', '.,uw,,', '_.dr_,', 'A'] | Prediccion D
Muestra ['____,,', '_uw._.', ',_.dr,', 'A'] | Prediccion D
Muestra [',_,__,', '.,uw._', '_.._dr', 'A'] | Prediccion D
Muestra [',..,,_', ',_,_uw', '.dr..,', 'A'] | Prediccion D
Top 10 false positives: 

Muestra ['.__.__', '.,w.__', '_._.r.', 'C'] | Prediccion V
Muestra [',._,._', ',.....', '_.r,_,', 'C'] | Prediccion V
Muestra [',_,_,,', '_w..,,', ',,,,r.', 'C'] | Prediccion D
Muestra ['.,....', 'wu..,.', '.,r,_.', 'C'] | Prediccion V
Muestra ['.__.__', ',.,,._', '...,r.', 'C'] | Prediccion V
Muestra [',.,.,.', '_.uw.,', ',__,r,', 'C'] | Prediccion V
Mues

# Training evolution

### CNN

In [ ]:
display_epochs_stats(cnn_epoch_stats, num_experiments, display_train_loss=False, display_val_loss=False)

### LSTM

In [ ]:
display_epochs_stats(lstm_epoch_stats, num_experiments, display_train_loss=False, display_val_loss=False)

### DeepSet

In [ ]:
display_epochs_stats(deepset_epoch_stats, num_experiments, display_train_loss=False, display_val_loss=False)

### Deepset V2

In [ ]:
display_epochs_stats(deepsetv2_epoch_stats, num_experiments, display_train_loss=False, display_val_loss=False)

### FeedFordward

In [ ]:
display_epochs_stats(feedforward_epoch_stats, num_experiments, display_train_loss=False, display_val_loss=False)